## Getting Set Up

### Import the necessary libraries

In [1]:
from requests import get, post
import pandas as pd

### API Keys
Add your API key here. We will pass it to any API calls we make through an header.

In [2]:
API_KEY = "YOUR_API_KEY"
HEADER = {"x-dune-api-key" : API_KEY}

### Simplifying URL generation
Let us create a function to generate different URLs for different API endpoints. You would see this in action in the coming section.

In [3]:
BASE_URL = "https://api.dune.com/api/v1/"

def make_api_url(module, action, ID):
    """
    We shall use this function to generate a URL to call the API.
    """
    
    url = BASE_URL + module + "/" + ID + "/" + action
    
    return url

### Wrapping API endpoints in functions

> The Dune API currently has four primary end points. We are going to wrap these up in neat functions. Which shall then make it very easy to use the API. As you would see in the coming sections.

In [4]:
def execute_query(query_id):
    """
    Takes in the query ID.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """
    
    url = make_api_url("query", "execute", query_id)
    response = post(url, headers=HEADER)
    execution_id = response.json()['execution_id']
    
    return execution_id


def get_query_status(execution_id):
    """
    Takes in an execution ID.
    Fetches the status of query execution using the API
    Returns the status response object
    """
    
    url = make_api_url("execution", "status", execution_id)
    response = get(url, headers=HEADER)
    
    return response


def get_query_results(execution_id):
    """
    Takes in an execution ID.
    Fetches the results returned from the query using the API
    Returns the results response object
    """
    
    url = make_api_url("execution", "results", execution_id)
    response = get(url, headers=HEADER)
    
    return response


def cancel_query_execution(execution_id):
    """
    Takes in an execution ID.
    Cancels the ongoing execution of the query.
    Returns the response object.
    """
    
    url = make_api_url("execution", "cancel", execution_id)
    response = get(url, headers=HEADER)
    
    return response

# Using the API

### Execute a Query
You can pass any query ID from Dune you would like to fetch data from.

In [5]:
execution_id = execute_query("1258228")

 > This function returned an execution ID. Which we shall now pass to rest of the APIs through our functions.

In [6]:
execution_id

'01GCX1HSVDPQN1SSQ93TR266KX'

### Get Query Status

In [7]:
response = get_query_status(execution_id)

 > In this response object, you might primarily want to check the `state` which in our case is `QUERY_STATE_COMPLETED`. Indicating the query has completed execution.

In [8]:
response.json()

{'execution_id': '01GCX1HSVDPQN1SSQ93TR266KX',
 'query_id': 1258228,
 'state': 'QUERY_STATE_COMPLETED',
 'submitted_at': '2022-09-14T03:44:14.204738Z',
 'expires_at': '2024-09-13T03:44:14.29315Z',
 'execution_started_at': '2022-09-14T03:44:14.210044031Z',
 'execution_ended_at': '2022-09-14T03:44:14.293148291Z',
 'result_metadata': {'column_names': ['block_time',
   'token_a_symbol',
   'token_b_symbol',
   'token_a_amount',
   'token_b_amount',
   'project',
   'version',
   'category',
   'trader_a',
   'trader_b',
   'token_a_amount_raw',
   'token_b_amount_raw',
   'usd_amount',
   'token_a_address',
   'token_b_address',
   'exchange_contract_address',
   'tx_hash',
   'tx_from',
   'tx_to',
   'trace_address',
   'evt_index',
   'trade_id'],
  'result_set_bytes': 5130,
  'total_row_count': 10,
  'datapoint_count': 220,
  'pending_time_millis': 5,
  'execution_time_millis': 83}}

### Get Query results
Finally we would like to load the results from the execution of our query.

In [9]:
response = get_query_results(execution_id)

Lets wrap this data up in a neat pandas dataframe

In [10]:
data = pd.DataFrame(response.json()['result']['rows'])

And here is your data from your Dune query. In a table. In Python. Hurray!

In [11]:
data.head()

,block_time,category,evt_index,exchange_contract_address,project,token_a_address,token_a_amount,token_a_amount_raw,token_a_symbol,token_b_address,...,token_b_symbol,trace_address,trade_id,trader_a,trader_b,tx_from,tx_hash,tx_to,usd_amount,version
0,2021-01-30T17:41:27+00:00,DEX,262,\x74948a441fa42ae60fc39e993ed0a81041186566,Uniswap,\x5af2be193a6abca9c8817001f45744777db30756,2125.028035,212502803498,VGX,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,WETH,[],1,\x19654fbe6b0d05a68c3fadb833ce1fdc66f98b77,None,\x19654fbe6b0d05a68c3fadb833ce1fdc66f98b77,\x86be9e16bef9807ece751ddffde6bb11aa5652cd6f25...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,4963.830524,2
1,2021-02-01T18:55:51+00:00,DEX,214,\x9f4aa9b4661f0c55b61fc12b1944f006a71c773f,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.958622,958622412961025500,WETH,\x8a9c4dfe8b9d8962b31e4e16f8321c44d48e246e,...,NCT,[],1,\x7a08ed5862ba4c2887ef169a17637eadb27beecf,None,\x979636bfeecd6a3ba720cf6eb8b7135f247e0b4a,\x86beabdd4b5b94a0d8438a414115da77eb2c5dcd0fb5...,\x7a08ed5862ba4c2887ef169a17637eadb27beecf,1268.075314,2
2,2021-01-30T22:54:23+00:00,DEX,5,\xc2adda861f89bbb333c90c492cb837741916a225,Uniswap,\x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,18.572913,18572912800361480000,MKR,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,WETH,[],1,\x4d246be90c2f36730bb853ad41d0a189061192d3,None,\x3a8e730807c58facb9a55f94bee7c28eb01b35f5,\x86beb23984b03bdda3a9894f49dcb7ad165679708853...,\x4d246be90c2f36730bb853ad41d0a189061192d3,29820.297334,2
3,2021-02-01T03:10:09+00:00,DEX,165,\x11b1f53204d03e5529f09eb3091939e4fd8c9cf3,Uniswap,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,7.963170,7963170431406298000,WETH,\x0f5d2fb29fb7d3cfee444a200298f468908cc942,...,MANA,[],1,\xd3d2e2692501a5c9ca623199d38826e513033a17,None,\x1d5bd81a1dfad3d32cb5c3e5cf04a30da5af710a,\x86becfffb8560137fc94ababe5cfc2130b75474bd68f...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,10469.657956,2
4,2021-02-01T03:10:09+00:00,DEX,168,\xd3d2e2692501a5c9ca623199d38826e513033a17,Uniswap,\x1f9840a85d5af5bf1d1762f925bdaddc4201f984,600.000000,600000000000000000000,UNI,\xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,WETH,[],1,\x1d5bd81a1dfad3d32cb5c3e5cf04a30da5af710a,None,\x1d5bd81a1dfad3d32cb5c3e5cf04a30da5af710a,\x86becfffb8560137fc94ababe5cfc2130b75474bd68f...,\x7a250d5630b4cf539739df2c5dacb4c659f2488d,10470.000000,2


#### If you like, you can also store this data in a CSV file.

In [13]:
data.to_csv("sample_data.csv")

### Cancel Query Execution

 > Some queries can take a long time to execute (minutes), and you might want to interrupt this execution at times. This is how to do that.

In [12]:
response = cancel_query_execution(execution_id)

When you have a running query and call this function. You would get a response object returned to you confirming the cancellation of query execution.

## Execute a Query with Parameters

In [45]:
def execute_query_with_params(query_id, param_dict):
    """
    Takes in the query ID. And a dictionary containing parameter values.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """
    
    url = make_api_url("query", "execute", query_id)
    response = post(url, headers=HEADER, json={"query_parameters" : param_dict})
    execution_id = response.json()['execution_id']
    
    return execution_id

### Create a dictionary of parameters you need to pass to the Dune parametrised Query
For our example, we are passing an `wallet_address`. And our query shall return to us the total amount spent on gas from this `wallet_address`.

In [46]:
parameters = {"wallet_address" : "0xb10f35351ff21bb81dc02d4fd901ac5ae34e8dc4"}

### Pass this Dictionary of parameters with the Execution End Point
Here we shall simply pass this to the function we have written to do this for us.

In [47]:
execution_id = execute_query_with_params("638435", parameters)

### Rest of the End Points remain the same
Once you have the Execution ID from calling this API end point. The usage with other three API endpoints (getting status, results and cancelling execution) remains the same, irrespective of weather you executed a query with or without parameters.